In [10]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

import torch
from torch_geometric.nn import GAE
from torch_geometric.utils import is_undirected, to_undirected


from torch.utils.data import DataLoader, TensorDataset

from model.GCN_GRU import GCN_GRU, Decoder
from utils import *

In [2]:
shop_col = 'stonc_6_label'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

embedding_dim = 64
entity_dim = 8
epochs = 1000
learning_rate = 0.0001
batch_size = 1000

weights_path = 'GCN_GRU_Link_Prediction'

In [3]:
data_path = './data'

chid_dict_file = 'sample/sample_50k_idx_map.npy'
cdtx_file = 'sample/sample_50k_cdtx.csv'
cust_file = 'preprocessed/df_cust_log_without_shop.csv'

chid_path = os.path.join(data_path, chid_dict_file)
cdtx_path = os.path.join(data_path, cdtx_file)
cust_path = os.path.join(data_path, cust_file)

In [4]:
df_cdtx, df_cust, n_users, n_shops = read_sample_files(cdtx_path,
                                                       cust_path,
                                                       chid_path,
                                                       shop_col)

Start reading cdtx file...
Finish reading cdtx file !
Start reading cust file...
Finish reading cust file !
Start maping encodeing...
Finish !!


In [12]:
list_months = sorted(df_cdtx.csmdt.unique())
ma = np.log(df_cdtx.groupby(['chid', 'csmdt']).objam.sum().max())

ignore_cols = ['chid', 'data_dt']
category_cols = [f'category_{i+1}' for i in range(6)]
numeric_cols = list(set(df_cust.columns) - set(category_cols) - set(ignore_cols))

emb_dims = list(df_cust[category_cols].nunique())

input_dim = len(category_cols)*entity_dim + len(numeric_cols)

In [7]:
edge_dict = {}
edge_weights_dict = {}
for month in tqdm(list_months):
    edges = df_cdtx[df_cdtx.csmdt==month].groupby(['chid', shop_col]).objam.sum()
    edge_pairs = np.stack([np.array(i) for i in edges.index]).T
    edge_pairs = torch.LongTensor(edge_pairs)
    
    edge_weights = np.log(edges.values+1)
    edge_weights = torch.Tensor(edge_weights)
    
    if not is_undirected(edge_pairs):
        edge_pairs = torch.cat([edge_pairs, edge_pairs[[1,0],:]], -1)
        edge_weights = edge_weights.repeat(2)
    
    edge_dict[month] = edge_pairs
    edge_weights_dict[month] = edge_weights

100%|██████████| 25/25 [00:18<00:00,  1.35it/s]


In [14]:
model = GAE(GCN_GRU(input_dim, embedding_dim, entity_dim, emb_dims), Decoder(embedding_dim, 6)).to(device)

x_num = []
x_cat = []
y = []
for i in list_months:
    cust_num_features = df_cust[df_cust.data_dt==i][numeric_cols].to_numpy()
    cust_num_features = torch.Tensor(cust_num_features)
    cust_cat_features = df_cust[df_cust.data_dt==i][category_cols].to_numpy()
    cust_cat_features = torch.LongTensor(cust_cat_features)
    
    shop_num_features = torch.zeros(n_shops, cust_num_features.shape[1])
    shop_cat_features = torch.zeros(n_shops, cust_cat_features.shape[1]).long()
    
    temp_y = np.e ** (df_cust[df_cust.data_dt==i]['objam'].to_numpy()*ma)
    ind1 = (temp_y >0) & (temp_y <= 1e4)
    ind2 = (temp_y >1e4) & (temp_y <= 5e4)
    ind3 = (temp_y >5e4) & (temp_y <= 1e5)
    ind4 = (temp_y >1e5) & (temp_y <= 3e5)
    ind5 = (temp_y >3e5)

    temp_y[ind1] = 1
    temp_y[ind2] = 2
    temp_y[ind3] = 3
    temp_y[ind4] = 4
    temp_y[ind5] = 5
    
    x_num.append(torch.cat([cust_num_features, shop_num_features], 0).to(device))
    x_cat.append(torch.cat([cust_cat_features, shop_cat_features], 0).to(device))
    y.append(torch.LongTensor(temp_y).to(device))
    
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
pos_edge_index = [i.to(device) for i in edge_dict.values()]
pos_edge_weigts = [i.to(device) for i in edge_weights_dict.values()]
criterion = torch.nn.CrossEntropyLoss()

In [15]:
def train():
    model.train()
    train_output = []
    train_y = np.array([])
    for i in tqdm(range(10)):
        
        train_dataset = TensorDataset(y[i+12])
        train_loader = DataLoader(dataset=train_dataset, shuffle=False, batch_size=batch_size)
        
        for j, true_y in enumerate(train_loader):
            optimizer.zero_grad()
            z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
            output = model.decode(z[j*batch_size:(j+1)*batch_size]).sigmoid()
            loss = criterion(output, true_y[0])
            train_output.append(output.cpu().detach().numpy())
            train_y = np.concatenate([train_y, true_y[0].cpu().detach().numpy().reshape(-1)])
        
            loss.backward(retain_graph=True)
            optimizer.step()
        
            
    return loss/10, train_output, train_y

In [16]:
def test():
    model.eval()
    test_output = []
    test_y = np.array([])
    
    for i in range(10,12):

        with torch.no_grad():
            z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
            output = model.decode(z[:n_users]).sigmoid()
            test_output.append(output.cpu().detach().numpy())
            test_y = np.concatenate([test_y, y[i+12].cpu().detach().numpy().reshape(-1)])
            
    return test_output, test_y

In [ ]:
for epoch in range(epochs):
    loss, train_output, train_y  = train()
    
    test_output, test_y = test()
    
    print('train',f1_score(train_y, np.argmax(np.concatenate(train_output,0),-1), average='macro'))
    print('test', f1_score(test_y, np.argmax(np.concatenate(test_output,0),-1), average='macro'))

 90%|█████████ | 9/10 [01:51<00:12, 12.42s/it]

In [ ]:
torch.save(model.encoder.state_dict(), weights_path)